#### Marathon Training Application Story Template
##### Objective: Create an engaging story template for a marathon training application that adapts to user prompts, running metrics, and training styles. This story template will be used to generate specific stories based on user inputs and metrics.

In [1]:
# Imports
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
import torch
import evaluate

# MPT-7B Instruct Model

In [2]:
model_id ="mosaicml/mpt-7b-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", trust_remote_code=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\pmahi\.cache\huggingface\modules\transformers_modules\mosaicml\mpt-7b-instruct\7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb\configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
C:\Users\pmahi\.cache\huggingface\modules\transformers_modules\mosaicml\mpt-7b-instruct\7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb\configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))


pytorch_model-00001-of-00002.bin:   3%|3         | 336M/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.36G [00:00<?, ?B/s]

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# Constants for prompt generation
INSTRUCTION_KEY = "### Instruction:"
RESPONSE_KEY = "### Response:"
END_KEY = "### End"
INSTRUCTION = """Can you help me write a story with the following details:\
Prompt: A zombie horror story set in New York City.
Distance: 0.5 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint \n
"""
INTRO_BLURB = """You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.
"""
input_text = """{intro}
{instruction_key}
{instruction}
{response_key}
""".format(
    intro=INTRO_BLURB,
    instruction_key=INSTRUCTION_KEY,
    instruction=INSTRUCTION,
    response_key=RESPONSE_KEY,
)
# print(input_text)

In [4]:
encoded_input = tokenizer(input_text, return_tensors='pt').to("cuda")
streamer = TextStreamer(tokenizer)
response = model.generate(input_ids=encoded_input.input_ids, streamer=streamer, max_new_tokens=1000, temperature=1, repetition_penalty=2.0, no_repeat_ngram_size=1).to("cuda")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Instruction:
Can you help me write a story with the following details:Prompt: A zombie horror story set in New York City.
Distance: 0.5 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint 


### Response:


C:\Users\pmahi\.cache\huggingface\modules\transformers_modules\mosaicml\mpt-7b-instruct\7bf8dfd6c819cdb82e2f9d0b251f79ddd33314fb\attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and passing that to each attention ' + 'module instead.')


Here is one possible draft of your requested short narrative, which I hope provides an interesting reading experience while also serving as useful guidance towards achieving desired fitness goals through regular exercise practice! The main characters here include two young adults from contrasting backgrounds - namely our protagonist 'Alex', whose privileged upbringing has afforded him many advantages throughout life but left his mental health somewhat fragile; alongside her best friend / roommate Emily whom Alex meets whilst attending college at NYU after being rescued amidst chaos & destruction caused across muchof Manhattan dueto recent outbreak or undead virus infection spreading rapidly amongst local population... 
   As they struggle together against overwhelming odds both physically (in terms fo their efforts t survive hunger& thirst)and mentally(as each must confront fears about personal safety), it becomes clear how important friendship can trulybe when facing such adversity --

In [5]:
print(tokenizer.decode(response[0], skip_special_tokens=True))

You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style.

### Instruction:
Can you help me write a story with the following details:Prompt: A zombie horror story set in New York City.
Distance: 0.5 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint 


### Response:
Here is one possible draft of your requested short narrative, which I hope provides an interesting reading experience while also serving as useful guidance towards achieving desired fitness goals through regular exercise practice! The main characters here include two young adults from contr

# Evaluate

In [6]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

In [7]:
# print(round(results["mean_perplexity"], 2))